# Retrive ES data for Bruppleinitial and Burpple
    save the object with pickle

    >'burpple__review__unique__***today***.p',   
    

## 1. Conenct with ES

In [1]:
%load_ext autoreload
%autoreload 2
import myfunctions as f
f.today()

'20180313'

In [2]:
# set up ElasticSearch object and the URL to access it
def setup_es(isServer):
    # python version
    import sys
    print "system_info: %s"%sys.version
    # current working directory
    import os
    print "path_info: %s"%os.getcwd()    
    ## Local on PC/laptop or on VM (10.0.106.122:2)   
    from elasticsearch import Elasticsearch
    port = "9200"
    host = "localhost"
    if isServer:
        host = "10.0.109.54"    
    url = "http://" + host + ":" + port    
    print "es_info: %s"%url    
    es = Elasticsearch([{'host': host, 'port': port}])
    return es, url

es, url = setup_es(True)

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712
es_info: http://10.0.109.54:9200


In [3]:
# make sure ES is up and running
import requests
def initialise_es(i):
    res = requests.get(url)
    if i:
        print(res.content)
initialise_es(1)

{
  "name" : "star7",
  "cluster_name" : "star",
  "cluster_uuid" : "VDcV69ThThyb00sd6PgeMA",
  "version" : {
    "number" : "2.4.5",
    "build_hash" : "c849dd13904f53e63e88efc33b2ceeda0b6a1276",
    "build_timestamp" : "2017-04-24T16:18:17Z",
    "build_snapshot" : false,
    "lucene_version" : "5.5.4"
  },
  "tagline" : "You Know, for Search"
}



In [4]:
# define dict for b and bi
burpple = {"_index":"burpple"}
burppleinitial = {"_index":"burppleinitial"}
burpples = [burpple, burppleinitial]
for website in burpples:
    website.update({"vendor":"restaurant",
                    "review":"review",  
                    "user":"user",                                   
                    "_id":"_id",
                    "vendor_id":'_source.id',
                    "vendor_url":'_source.url',
                    "vendor_name":"_source.name", 
                    "address":"_source.address", 
                    "cuisine_tags":"_source.tags", 
                    "phone":"_source.phone",                
                    "cycleStart":'_source.crawlTimeStamp'})    

In [5]:
burpple.update({"reviewFeedTime":'_source.feedDatetime',})
burppleinitial.update({"reviewFeedTime":'_source.datetime',})

## 2. Actual data retrival

In [6]:
def retrive_data (website, doc_type):
    # activate ES
    initialise_es(0)
    # ES search pattern
    _body = {"query": {"match_all": {}}}
    _index = website['_index']
    _doc_type = website[doc_type]

    # With the help of a generator, get all records
    from elasticsearch import helpers
    scanResp = helpers.scan(es, _body, scroll= "2m", 
                            index= _index, 
                            doc_type= _doc_type, 
                            timeout="2m")
    recs = []
    for rec in scanResp:
        recs.append(rec)
    print len(recs)

    # Convert unicode to string (ascii, ignore unicode such as '\xae')
    def convert_unicode(data):
        if isinstance(data, basestring):
            return (data.encode("ascii","ignore"))
        elif isinstance(data, collections.Mapping):
            return dict(map(convert_unicode, data.iteritems()))
        elif isinstance(data, collections.Iterable):
            return type(data)(map(convert_unicode, data))
        else:
            return data

    # json file to dataframe 
    import collections
    import pandas as pd
    pd.options.mode.chained_assignment = None # default is warn
    from pandas.io.json import json_normalize
    lst_rec = []
    for line in recs:
        line = convert_unicode(line)
        lst_rec.append(json_normalize(line))    
    df = pd.concat(lst_rec) 
    # Print shape and all attributes
    print ("ES location: %s, %s"%(_index,website[doc_type]))
    print("Dimention: %d , %d"%df.shape)
    print("Column names::%s"%", ".join(df.columns.tolist()))
    print ("\n")
    
    return df

In [7]:
%%time
initialise_es(1)
print ""

website = burpple
website.update({"all_rec_review" : retrive_data (website, "review")})

{
  "name" : "star7",
  "cluster_name" : "star",
  "cluster_uuid" : "VDcV69ThThyb00sd6PgeMA",
  "version" : {
    "number" : "2.4.5",
    "build_hash" : "c849dd13904f53e63e88efc33b2ceeda0b6a1276",
    "build_timestamp" : "2017-04-24T16:18:17Z",
    "build_snapshot" : false,
    "lucene_version" : "5.5.4"
  },
  "tagline" : "You Know, for Search"
}


25709
ES location: burpple, review
Dimention: 25709 , 43
Column names::_id, _index, _score, _source.body, _source.crawlTimeStamp, _source.feedDatetime, _source.foodImgUrl, _source.id, _source.name, _source.postDatetime, _source.restaurant, _source.restaurant.address, _source.restaurant.areaUrls, _source.restaurant.avgPrice, _source.restaurant.crawlTimeStamp, _source.restaurant.id, _source.restaurant.name, _source.restaurant.openingHours, _source.restaurant.phone, _source.restaurant.tags, _source.restaurant.url, _source.restaurant.website, _source.restaurantUrl, _source.title, _source.url, _source.user, _source.user.awards, _source.user.badg

In [8]:
%%time
website = burppleinitial
website.update({"all_rec_review" : retrive_data (website, "review")})

297480
ES location: burppleinitial, review
Dimention: 297480 , 44
Column names::_id, _index, _score, _source.areaName, _source.body, _source.crawlTimeStamp, _source.datetime, _source.foodImgUrl, _source.id, _source.name, _source.offset, _source.restaurant, _source.restaurant.address, _source.restaurant.areaUrls, _source.restaurant.avgPrice, _source.restaurant.crawlTimeStamp, _source.restaurant.id, _source.restaurant.name, _source.restaurant.openingHours, _source.restaurant.phone, _source.restaurant.tags, _source.restaurant.url, _source.restaurant.website, _source.restaurantUrl, _source.title, _source.url, _source.user, _source.user.awards, _source.user.badges, _source.user.bio, _source.user.country, _source.user.crawlTimeStamp, _source.user.name, _source.user.numOfFollowers, _source.user.numOfFollowing, _source.user.numOfReviews, _source.user.numOfWishlists, _source.user.url, _source.user.userImgUrl, _source.user.username, _source.user.website, _source.userImgUrl, _source.username, _ty

## 3. Duplicate records removal  

In [9]:
def remove_bu_dup_records(burpples, non_sg_vendors):
    # not included: '_source.url', # https://www.burpple.com/f/ + "_id"
    cols = ['_id', # review identifier, something like "liKrL-pE"
            '_type',
            '_index', # burpple / burpple initial
            '_source.title', # title of review, with some special characters
            '_source.body', # text 
            '_source.crawlTimeStamp', 
            '_source.foodImgUrl',             
            '_source.username', # user identifier
            '_source.restaurant.id',
            '_source.restaurant.name'] #vendor identifier
    # merge feed time
    import pandas as pd
    pd.options.mode.chained_assignment = None # default is warn
    df = pd.concat([site["all_rec_review"]
                    [cols+ [site["reviewFeedTime"]]] for site in burpples])
    df["feedTime"] = df[[site["reviewFeedTime"] for site in burpples]].fillna('').sum(axis=1)
    df = df[cols +["feedTime"]]
    # sort, only leaving the latest crawled first
    df = df.sort_values(by=['_source.crawlTimeStamp',"_id"], ascending=[False,True])
    df = df.groupby("_id").first()
    df.reset_index(inplace=True)
    # remove reviews of non-sg vendors
    df = df[~df["_source.restaurant.id"].isin(non_sg_vendors)]
    
    print ("Got %d unique records of review" % len(df))
    print ("Got %d unique records of images" % 
           df[df['_source.foodImgUrl']!=""]['_source.foodImgUrl'].nunique())
    print ("Got %d unique users" % df['_source.username'].nunique())
    print ("Got %d unique vendors" % df['_source.restaurant.id'].nunique())
    print ("Review time from %s to %s" % (df['feedTime'].min(),df['feedTime'].max()))
    return df
    

In [10]:
%%time
non_sg_vendors = ['114803', '119954', '136868', '139058', '149618', '155202', '156512', '16431',
                  '165933', '166041', '174029', '28318', '43464', '51131', '59732', '63212']
df = remove_bu_dup_records(burpples, non_sg_vendors)

Got 323104 unique records of review
Got 323051 unique records of images
Got 11180 unique users
Got 14547 unique vendors
Review time from 2011-11-08 00:00:00.000000 to 2017-07-20 23:04:45.837391
CPU times: user 8.06 s, sys: 276 ms, total: 8.33 s
Wall time: 8.33 s


In [11]:
df.head(3)

,_id,_type,_index,_source.title,_source.body,_source.crawlTimeStamp,_source.foodImgUrl,_source.username,_source.restaurant.id,_source.restaurant.name,feedTime
0,--0EG2cp,review,burppleinitial,Sicilian Lemon Creme andCaramelized Red Mixed ...,,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/376d87a944deb0499...,@ieatkows,34166,OSO Ristorante,2014-12-22 00:00:00.000000
1,--0ck5YP,review,burppleinitial,Just look at their soy sauce chicken.,The tender and juicy thigh meat with their fra...,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/2fbab5e890cf97f31...,@jiaknonstop,164762,Fatty Ox Hong Kong Kitchen (Chinatown Complex ...,2014-12-15 00:00:00.000000
2,--0gMGRO,review,burppleinitial,"Holy, smokin' Burnt Corn Tacos - it's their WE...",(Read: time for MORE alcohol.) #AntiDoteBar #...,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/16ff778f20e5f8614...,@Jazpster,539,Asian Market Caf (Fairmont Singapore),2016-11-18 00:00:00.000000


## 4. Save Files

In [13]:
%%time
file_name = "burpple__review__unique__"+f.today()+".p"
f.save_files(df, file_name)

burpple__review__unique__20180313.p
CPU times: user 17.5 s, sys: 452 ms, total: 18 s
Wall time: 18 s


## Further cleaning

In [1]:
%load_ext autoreload
%autoreload 2
import myfunctions as f
f.today()

'20180317'

In [2]:
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712


In [3]:
file_name = "burpple__review__unique__20180313.p"    
df = f.retrive_file(file_name)

retrived: burpple__review__unique__20180313.p


## 3. Obtain review objects with clean text

In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None # default is warn
# new col: loc
df["loc"] = df["_index"] + "/" + df["_type"] + "/" + df["_id"] 
cols = ['loc','_source.title','_source.body','_source.restaurant.name']
df = df[cols]
rename_cols = [col.replace("_source.", "") for col in cols]
df.columns = rename_cols
# new col: full_review
df["full_review"] = df['title'] + " " + df['body']

In [5]:
""" 
twitter style text-cleaning, for url, mention, hashtag, 
also: dollar, score
not: apostrophe conversion, stop words, emoticons, slang, word standardization
"""
def clean_review(s):
    import re   
    # 1. etract urls (may contain $, #, @), remove url part from s
    p = r'http[s]?://(?:[a-z]|[0-9]|[$-_@#.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
    urls = tuple(re.findall(p, s))
    for rep in urls:
        s = s.replace(rep,"")
    # 2. normal cleaning: replace w/ and &
    s = " " + s +" "
    s = s.replace("w/", " with ")
    s = s.replace("&", " and ") 
    # 3. textract hashtags 
    hashtags = tuple(re.findall(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", s))
    # 4. extract mentions
    mentions = tuple(re.findall(r'(?:@[\w_]+)', s))   
    # 5. extract dolalr amounts
    dollars = tuple([x[0] for x in re.findall(r'(\$\d+([,\.]\d+)?(\+\+)?(\+)?k?)', s)])
    # 6. extract scores
    scores = tuple([x[0] for x in re.findall(r'(\d+([\.]\d+)(\/)\d+([\.]\d+)?)', s)])
    # 7. remove above parts from review, strip and remove multiple space
    words = sorted(list(dollars + scores + hashtags + mentions), key=len, reverse=True)
    for rep in words:
        s = s.replace(rep,"")
    s = re.sub(' +',' ', s.strip()) 
    lst = [s, hashtags, mentions, dollars, scores, urls]
    return (tuple(lst))

In [6]:
%%time
df["review_text"], df["hashtags"], df["mentions"], df["dollars"], df["scores"], df["urls"] = \
zip(*df["full_review"].apply(clean_review))
cols = df.columns.tolist()
cols.remove("full_review")
df = df[cols]

CPU times: user 15.3 s, sys: 652 ms, total: 16 s
Wall time: 15.8 s


In [7]:
%%time
# On review text, perform clean_name_v2()
df["clean_review"] = df["review_text"].apply(f.clean_name_v2)

CPU times: user 22.8 s, sys: 56 ms, total: 22.9 s
Wall time: 22.9 s


In [8]:
print df.shape
print df.columns.tolist()
df.sample(2,random_state=2008)

(323104, 11)
['loc', 'title', 'body', 'restaurant.name', 'review_text', 'hashtags', 'mentions', 'dollars', 'scores', 'urls', 'clean_review']


,loc,title,body,restaurant.name,review_text,hashtags,mentions,dollars,scores,urls,clean_review
242662,burppleinitial/review/jnF_zvaY,"Aged Cheddar, Fig & #Cognac Jam and Candied Wa...","Aged Cheddar, Fig & #Cognac Jam and Candied Wa...",The Humble Loaf,"Aged Cheddar, Fig and Jam and Candied Walnuts ...","(#Cognac, #lunch, #weekend, #Cognac, #lunch, #...",(),(),(),(),aged cheddar fig and jam and candied walnuts o...
74846,burppleinitial/review/CzDetUln,Orange Brioche,,Artisan Boulangerie Co. (Great World City),Orange Brioche,(),(),(),(),(),orange brioche


In [9]:
%%time
file_name = "burpple__review__clean__"+f.today()+".p"
f.save_file(df, file_name)

saved: burpple__review__clean__20180317.p
CPU times: user 17.3 s, sys: 632 ms, total: 17.9 s
Wall time: 17.9 s


In [10]:
all_reviews = sorted([s for s in set(df["clean_review"].tolist()) if len(s)>0])
print "number of unique reviews: %d"%len(all_reviews)

number of unique reviews: 277772


** retrive food items**

In [13]:
file_name = "food_entities__20180317.p"
food_entities = f.retrive_file(file_name)

retrived: food_entities__20180317.p


In [22]:
food_entities = food_entities.reset_index()
# remove food of " and "-PRON-"
food_entities = food_entities[~food_entities["assigned_name"].isin(["", "-PRON-"])]
# food name variant : food_item_index
index_ref = dict()
def add_ref(line):
    v = line["index"]
    index_ref.update({k:v for k in line["name_variants"]})
food_entities.apply(add_ref, axis =1)
food_items = index_ref.keys()
print "number of food searched for: %d"%len(food_items)

number of food searched for: 139083


In [23]:
file_name = "words_to_remove_20180308.csv"
rmv = pd.read_csv(file_name, header=None)[0].tolist()
rmv = rmv+['can', 'sweet', 'love', 'up', 'no', 'me', 'back', 'fluffy', 'hot', 'spicy', 'crisp', 'salty', 'house', 'red', 'fat', 'roasted', 'white', 'refreshing', 'black', 'cold', 'deep fried', 'pieces', 'toasted', 'skin', 'cheesy', 'hungry', 'pop', 'miss', 'option', 'fried', 'tomato', 'vanilla', 'plus', 'potato', 'thai', 'spice', 'infused', 'sauces', 'to share', 'lover', 'surprise', 'balanced', 'choice', 'french', 'green', 'steamed', 'italian', 'wake up', 'silver', 'savoury flavours', 'western', 'set for', 'pan fried', 'the original', 'decadence', 'the great', 'uplifting', 'all in', 'meaty', 'spice level', 'per slice', 'discovery', 'smokey', 'all time favourites', 'sticky', 'the double', 'pint', 'mad', 'singapore style', 'alcoholic', 'serves', 'nutty', 'salted', 'lunch set', 'boost', 'anything', 'dips', 'vietnamese', 'set lunch', 'lager', 'for pax', 'below', 'bundle']

In [27]:
def search_food(review):
    return [s for s in f.search_food(review, values=food_items) if s not in rmv]

In [28]:
%%time
df_sample = df.sample(1000, random_state=2008)
df_sample["food_items"] = df_sample["clean_review"].apply(search_food)

CPU times: user 52 s, sys: 276 ms, total: 52.2 s
Wall time: 51.9 s


In [ ]:
df_sample.head(2)

,loc,title,body,restaurant.name,review_text,hashtags,mentions,dollars,scores,urls,clean_review,food_items
242662,burppleinitial/review/jnF_zvaY,"Aged Cheddar, Fig & #Cognac Jam and Candied Wa...","Aged Cheddar, Fig & #Cognac Jam and Candied Wa...",The Humble Loaf,"Aged Cheddar, Fig and Jam and Candied Walnuts ...","(#Cognac, #lunch, #weekend, #Cognac, #lunch, #...",(),(),(),(),aged cheddar fig and jam and candied walnuts o...,"[toast, jam, walnuts]"
74846,burppleinitial/review/CzDetUln,Orange Brioche,,Artisan Boulangerie Co. (Great World City),Orange Brioche,(),(),(),(),(),orange brioche,[orange brioche]


In [ ]:
%%time
"""
CPU times: user 4h 37min 15s, sys: 58.5 s, total: 4h 38min 14s
Wall time: 4h 37min 12s
"""
if 1==1:
    df["food_items"] = df["clean_review"].apply(search_food)
    file_name = "burpple__review__food__"+f.today()+".p" 
    f.save_file(df, file_name)

In [ ]:
df.sample(20, random_state=2008)